In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from CustomDatasetClass import * # EEG_Dataset, get_train_test_set()
from Models import * # CNN 
from TrainTestModule import * # train_model(), test_model()

In [2]:
torch.cuda.is_available()

True

### Setting
* lin_len 지우고, 자동 계산으로 바꾸기..**

In [3]:
# dataset setting ===================
data_type = "fftMap" # "fftMap", "fd"
dataset = "BCI" # "DEAP", "SEED", "BCI"
split = "6sec_5over" # window sliding
n_channel = 1 # input depth
n_electrodes = 8 # (num of eeg sensor's electrodes)
numOfClass = 4 # num of label

# label_names = ["low", "high"]
# label_names = ["low", "neutral", "high"]
label_mapping = {"neutral" : 0, "happy" : 1, "sad" : 2, "disgust" : 3}
label_names = list(label_mapping.keys())
print(label_names)
# ==================================

# model parameter ==================
model_type ="cla" # "reg"
lin_len = 884 if n_electrodes == 32 else 592 # vector's length after flattening # => 아 이건 계산하는걸로 바꾸자;
out_len = 1 if model_type == "reg" else numOfClass
# ===================================

bs = 64 # batch size

# not used for SEED dataset
target = 0 # (V=0, A=1, D=2, L=3)

['neutral', 'happy', 'sad', 'disgust']


### Load dataset

In [4]:
import pickle

data_path = "InputData/" + dataset + "_" + data_type + "_" + split + ".pickle"
meta_path = "InputData/" + dataset + "_" + split

with open(data_path, 'rb') as f:
    eeg_dataset = pickle.load(f)

if dataset == "SEED" or dataset == "BCI":
    with open(meta_path + "_ptc_to_idx", 'rb') as f:
        ptc_to_idx = pickle.load(f)
    with open(meta_path + "_idx_to_ptc", 'rb') as f:
        idx_to_ptc = pickle.load(f)

In [5]:
print("전체 데이터 개수 = ", len(eeg_dataset))
print("데이터 하나의 shape = ", eeg_dataset[0][0].shape)
print("레이블 example = ", eeg_dataset[0][1])

전체 데이터 개수 =  1894
데이터 하나의 shape =  (8, 41)
레이블 example =  sad


#### Label processing

* if value > 5  then class = 1(High)
* if value <= 5 then class = 0(Low)

In [6]:
if model_type == "cla": 
    for i in range(len(eeg_dataset)):
        if dataset == "DEAP":
            V, A, D, L = eeg_dataset[i][1]
            V = process_label(V, numOfClass); D = process_label(D, numOfClass); 
            A = process_label(A, numOfClass); L = process_label(L, numOfClass);
            eeg_dataset[i][1] = [V,A,D,L]
        elif dataset == "SEED": # already integer value
            eeg_dataset[i][1] += 1
        elif dataset == "BCI":
            eeg_dataset[i][1] = label_mapping[eeg_dataset[i][1]]

#### Traverse method
* DEAP도 ptc_to_idx, idx_to_ptc 파일을 만들자,,코드 개더럽;; **

In [7]:
if dataset == "DEAP":
    noc = len(eeg_dataset)//1280 # num of chunks
    print(noc)

    def get_data_index(p_id, t_id, c_id=0, noc=1):
        return (noc * 40) * p_id + noc * t_id + c_id

    def get_base_index(p_id, t_id):
        return 40 * p_id + t_id

    # get p_id, t_id, c_id
    def from_data_index(index, noc=1):
        p_id,remain = index // (noc*40), index % (noc*40)
        return p_id, remain//noc, remain%noc

elif dataset == "SEED" or dataset == "BCI":
    def get_data_index(p_id, t_id, c_id=0, noc=0): # noc is not used
        return ptc_to_idx[str(p_id+1) + "-" + str(t_id) + "-" + str(c_id)]

    # get p_id, t_id, c_id
    def from_data_index(index, noc=0): # noc is not used
        return idx_to_ptc[index]

In [8]:
# Basic train-test split
from sklearn.model_selection import train_test_split

In [9]:
def LOOCV_split(test_subject, eeg_dataset, target=0, noc=0):
    # test_subject = Leave this participant out
    train = []; test = []
    
    for idx in range(len(eeg_dataset)): 
        p_id, t_id, c_id = from_data_index(idx, noc) # *
        
        if p_id == test_subject:
            test.append(eeg_dataset[idx])
        else:
            train.append(eeg_dataset[idx])
    return train, test

In [20]:
test_subject = 0
train, test = LOOCV_split(test_subject, eeg_dataset) # LOOCV
print(len(train))
print(len(test))

1545
349


In [14]:
import torch
# 1. get neural network
net = CNN(n_channel, lin_len, out_len, n_electrodes, model_type)

# 2. train : test split
#train, test = LOOCV_split(test_subject, eeg_dataset) # LOOCV
train, test = train_test_split(eeg_dataset, test_size=0.2) # 8 : 2

# get dataset instance
trainset, trainloader, testset, testloader = get_train_test_set(train, test, bs, dataset)

# training
net, t_loss_list, v_loss_list, f1_list, acc_list = train_model(net, model_type, trainloader, testloader, label_names, epoch=400, detail=True)


[ Epoch 1 ] T_Loss = 0.021861 
      t/p   neutral   happy     sad disgust 
    neutral   114.0     0.0     0.0     0.0 
      happy    88.0     0.0     0.0     0.0 
        sad   111.0     0.0     0.0     0.0 
    disgust    66.0     0.0     0.0     0.0 
V_loss = 0.021777  Acc = 30.0792 

[ Epoch 2 ] T_Loss = 0.021804 
      t/p   neutral   happy     sad disgust 
    neutral   114.0     0.0     0.0     0.0 
      happy    88.0     0.0     0.0     0.0 
        sad   111.0     0.0     0.0     0.0 
    disgust    66.0     0.0     0.0     0.0 
V_loss = 0.021733  Acc = 30.0792 

[ Epoch 3 ] T_Loss = 0.021755 
      t/p   neutral   happy     sad disgust 
    neutral   114.0     0.0     0.0     0.0 
      happy    88.0     0.0     0.0     0.0 
        sad   111.0     0.0     0.0     0.0 
    disgust    66.0     0.0     0.0     0.0 
V_loss = 0.021688  Acc = 30.0792 

[ Epoch 4 ] T_Loss = 0.021712 
      t/p   neutral   happy     sad disgust 
    neutral   114.0     0.0     0.0     0.0 
     

      t/p   neutral   happy     sad disgust 
    neutral    91.0     0.0    21.0     2.0 
      happy    54.0     0.0    25.0     9.0 
        sad    12.0     0.0    98.0     1.0 
    disgust    31.0     0.0     6.0    29.0 
V_loss = 0.018985  Acc = 57.5198 

[ Epoch 30 ] T_Loss = 0.018659 
      t/p   neutral   happy     sad disgust 
    neutral    92.0     0.0    20.0     2.0 
      happy    54.0     0.0    26.0     8.0 
        sad    14.0     0.0    96.0     1.0 
    disgust    32.0     0.0     5.0    29.0 
V_loss = 0.018886  Acc = 57.2559 

[ Epoch 31 ] T_Loss = 0.018586 
      t/p   neutral   happy     sad disgust 
    neutral    95.0     0.0    17.0     2.0 
      happy    54.0     0.0    27.0     7.0 
        sad    13.0     0.0    97.0     1.0 
    disgust    32.0     0.0     5.0    29.0 
V_loss = 0.018797  Acc = 58.3113 

[ Epoch 32 ] T_Loss = 0.018451 
      t/p   neutral   happy     sad disgust 
    neutral    89.0     0.0    22.0     3.0 
      happy    53.0     0.0    26.


[ Epoch 58 ] T_Loss = 0.016237 
      t/p   neutral   happy     sad disgust 
    neutral    96.0     5.0    13.0     0.0 
      happy    19.0    42.0    16.0    11.0 
        sad     8.0     4.0    98.0     1.0 
    disgust    21.0    12.0     2.0    31.0 
V_loss = 0.016930  Acc = 70.4485 

[ Epoch 59 ] T_Loss = 0.016184 
      t/p   neutral   happy     sad disgust 
    neutral    92.0     8.0    14.0     0.0 
      happy    17.0    44.0    13.0    14.0 
        sad     8.0     4.0    98.0     1.0 
    disgust    19.0    14.0     1.0    32.0 
V_loss = 0.016909  Acc = 70.1847 

[ Epoch 60 ] T_Loss = 0.016144 
      t/p   neutral   happy     sad disgust 
    neutral    89.0     8.0    17.0     0.0 
      happy    17.0    44.0    13.0    14.0 
        sad     8.0     4.0    98.0     1.0 
    disgust    18.0    15.0     1.0    32.0 
V_loss = 0.016894  Acc = 69.3931 

[ Epoch 61 ] T_Loss = 0.016090 
      t/p   neutral   happy     sad disgust 
    neutral    89.0     8.0    17.0     0.0 
 

      t/p   neutral   happy     sad disgust 
    neutral   100.0     5.0     8.0     1.0 
      happy    18.0    45.0    12.0    13.0 
        sad     9.0     4.0    97.0     1.0 
    disgust    24.0    10.0     0.0    32.0 
V_loss = 0.016406  Acc = 72.2955 

[ Epoch 87 ] T_Loss = 0.015359 
      t/p   neutral   happy     sad disgust 
    neutral    91.0     7.0    15.0     1.0 
      happy    12.0    50.0    13.0    13.0 
        sad     9.0     4.0    97.0     1.0 
    disgust    17.0    15.0     1.0    33.0 
V_loss = 0.016383  Acc = 71.5040 

[ Epoch 88 ] T_Loss = 0.015335 
      t/p   neutral   happy     sad disgust 
    neutral    95.0     7.0    11.0     1.0 
      happy    15.0    48.0    12.0    13.0 
        sad     9.0     4.0    97.0     1.0 
    disgust    20.0    13.0     0.0    33.0 
V_loss = 0.016368  Acc = 72.0317 

[ Epoch 89 ] T_Loss = 0.015317 
      t/p   neutral   happy     sad disgust 
    neutral    93.0     7.0    13.0     1.0 
      happy    13.0    49.0    13.


[ Epoch 115 ] T_Loss = 0.014760 
      t/p   neutral   happy     sad disgust 
    neutral    99.0     7.0     6.0     2.0 
      happy     9.0    60.0     8.0    11.0 
        sad     9.0     4.0    97.0     1.0 
    disgust    21.0    11.0     0.0    34.0 
V_loss = 0.015898  Acc = 76.5172 

[ Epoch 116 ] T_Loss = 0.014749 
      t/p   neutral   happy     sad disgust 
    neutral    97.0     7.0     8.0     2.0 
      happy    10.0    59.0     8.0    11.0 
        sad     8.0     4.0    98.0     1.0 
    disgust    21.0    11.0     0.0    34.0 
V_loss = 0.015900  Acc = 75.9894 

[ Epoch 117 ] T_Loss = 0.014733 
      t/p   neutral   happy     sad disgust 
    neutral    97.0     7.0     8.0     2.0 
      happy     8.0    59.0     9.0    12.0 
        sad     8.0     4.0    97.0     2.0 
    disgust    18.0    14.0     0.0    34.0 
V_loss = 0.015886  Acc = 75.7256 

[ Epoch 118 ] T_Loss = 0.014720 
      t/p   neutral   happy     sad disgust 
    neutral    97.0     7.0     8.0     2.

...


[ Epoch 283 ] T_Loss = 0.013123 
      t/p   neutral   happy     sad disgust 
    neutral   103.0     3.0     6.0     2.0 
      happy     9.0    69.0     7.0     3.0 
        sad     5.0     2.0   103.0     1.0 
    disgust    15.0    11.0     0.0    40.0 
V_loss = 0.014568  Acc = 83.1135 

[ Epoch 284 ] T_Loss = 0.013115 
      t/p   neutral   happy     sad disgust 
    neutral   101.0     3.0     6.0     4.0 
      happy     6.0    71.0     8.0     3.0 
        sad     4.0     1.0   105.0     1.0 
    disgust    15.0    11.0     0.0    40.0 
V_loss = 0.014566  Acc = 83.6412 

[ Epoch 285 ] T_Loss = 0.013120 
      t/p   neutral   happy     sad disgust 
    neutral   102.0     3.0     6.0     3.0 
      happy     9.0    68.0     8.0     3.0 
        sad     4.0     2.0   104.0     1.0 
    disgust    15.0    11.0     0.0    40.0 
V_loss = 0.014564  Acc = 82.8496 

[ Epoch 286 ] T_Loss = 0.013124 
      t/p   neutral   happy     sad disgust 
    neutral   101.0     3.0     6.0     4.


[ Epoch 311 ] T_Loss = 0.013037 
      t/p   neutral   happy     sad disgust 
    neutral   103.0     3.0     6.0     2.0 
      happy     9.0    69.0     7.0     3.0 
        sad     6.0     2.0   102.0     1.0 
    disgust    16.0    10.0     0.0    40.0 
V_loss = 0.014531  Acc = 82.8496 

[ Epoch 312 ] T_Loss = 0.013056 
      t/p   neutral   happy     sad disgust 
    neutral   102.0     3.0     6.0     3.0 
      happy     6.0    71.0     8.0     3.0 
        sad     5.0     1.0   104.0     1.0 
    disgust    16.0     9.0     0.0    41.0 
V_loss = 0.014515  Acc = 83.9050 

[ Epoch 313 ] T_Loss = 0.013025 
      t/p   neutral   happy     sad disgust 
    neutral    99.0     4.0     6.0     5.0 
      happy     5.0    72.0     8.0     3.0 
        sad     4.0     2.0   104.0     1.0 
    disgust    15.0    10.0     0.0    41.0 
V_loss = 0.014522  Acc = 83.3773 

[ Epoch 314 ] T_Loss = 0.013034 
      t/p   neutral   happy     sad disgust 
    neutral    99.0     4.0     6.0     5.


[ Epoch 339 ] T_Loss = 0.012952 
      t/p   neutral   happy     sad disgust 
    neutral   100.0     3.0     6.0     5.0 
      happy     6.0    71.0     8.0     3.0 
        sad     6.0     2.0   102.0     1.0 
    disgust    15.0    10.0     0.0    41.0 
V_loss = 0.014481  Acc = 82.8496 

[ Epoch 340 ] T_Loss = 0.012966 
      t/p   neutral   happy     sad disgust 
    neutral   101.0     3.0     6.0     4.0 
      happy     7.0    70.0     8.0     3.0 
        sad     7.0     2.0   101.0     1.0 
    disgust    16.0     9.0     0.0    41.0 
V_loss = 0.014495  Acc = 82.5858 

[ Epoch 341 ] T_Loss = 0.012954 
      t/p   neutral   happy     sad disgust 
    neutral   101.0     3.0     6.0     4.0 
      happy     6.0    71.0     8.0     3.0 
        sad     6.0     2.0   102.0     1.0 
    disgust    16.0     9.0     0.0    41.0 
V_loss = 0.014479  Acc = 83.1135 

[ Epoch 342 ] T_Loss = 0.012969 
      t/p   neutral   happy     sad disgust 
    neutral    99.0     4.0     6.0     5.


[ Epoch 367 ] T_Loss = 0.012917 
      t/p   neutral   happy     sad disgust 
    neutral   100.0     3.0     6.0     5.0 
      happy     5.0    71.0     8.0     4.0 
        sad     6.0     2.0   102.0     1.0 
    disgust    14.0    10.0     1.0    41.0 
V_loss = 0.014458  Acc = 82.8496 

[ Epoch 368 ] T_Loss = 0.012911 
      t/p   neutral   happy     sad disgust 
    neutral   101.0     3.0     6.0     4.0 
      happy     7.0    71.0     8.0     2.0 
        sad     6.0     1.0   103.0     1.0 
    disgust    16.0     9.0     0.0    41.0 
V_loss = 0.014444  Acc = 83.3773 

[ Epoch 369 ] T_Loss = 0.012917 
      t/p   neutral   happy     sad disgust 
    neutral   101.0     3.0     6.0     4.0 
      happy     6.0    71.0     8.0     3.0 
        sad     6.0     2.0   102.0     1.0 
    disgust    16.0     9.0     0.0    41.0 
V_loss = 0.014451  Acc = 83.1135 

[ Epoch 370 ] T_Loss = 0.012933 
      t/p   neutral   happy     sad disgust 
    neutral   101.0     3.0     6.0     4.


[ Epoch 395 ] T_Loss = 0.012889 
      t/p   neutral   happy     sad disgust 
    neutral   100.0     3.0     6.0     5.0 
      happy     4.0    72.0     8.0     4.0 
        sad     6.0     2.0   102.0     1.0 
    disgust    14.0    10.0     1.0    41.0 
V_loss = 0.014425  Acc = 83.1135 

[ Epoch 396 ] T_Loss = 0.012898 
      t/p   neutral   happy     sad disgust 
    neutral   102.0     3.0     5.0     4.0 
      happy     7.0    71.0     8.0     2.0 
        sad     6.0     2.0   102.0     1.0 
    disgust    16.0     9.0     0.0    41.0 
V_loss = 0.014431  Acc = 83.3773 

[ Epoch 397 ] T_Loss = 0.012894 
      t/p   neutral   happy     sad disgust 
    neutral   100.0     3.0     6.0     5.0 
      happy     5.0    71.0     8.0     4.0 
        sad     6.0     1.0   103.0     1.0 
    disgust    14.0    10.0     1.0    41.0 
V_loss = 0.014436  Acc = 83.1135 

[ Epoch 398 ] T_Loss = 0.012892 
      t/p   neutral   happy     sad disgust 
    neutral   101.0     3.0     6.0     4.

### Leave-one-out-cross-validation over all participants

* 자살검토

In [18]:
numOfParticipants = 8

for test_subject in range(0, numOfParticipants):
    print(test_subject, "에 대한 LOOCV 학습")
    
    net = CNN(n_channel, lin_len, out_len, n_electrodes, model_type)
    
    train, test = LOOCV_split(test_subject, eeg_dataset)
    trainset, trainloader, testset, testloader = get_train_test_set(train, test, bs, dataset)
    net, t_loss_list, v_loss_list, f1_list, acc_list = train_model(net, model_type, trainloader, testloader, label_names, epoch=100, detail=True)

In [22]:
import torch
# 1. get neural network
net = CNN(n_channel, lin_len, out_len, n_electrodes, model_type)

# 2. train : test split
train, test = LOOCV_split(0, eeg_dataset) # LOOCV

# get dataset instance
trainset, trainloader, testset, testloader = get_train_test_set(train, test, bs, dataset)

# training
net, t_loss_list, v_loss_list, f1_list, acc_list = train_model(net, model_type, trainloader, testloader, label_names, epoch=50, detail=True)


[ Epoch 1 ] T_Loss = 0.022479 
      t/p   neutral   happy     sad disgust 
    neutral   116.0     0.0     0.0     0.0 
      happy    55.0     0.0     0.0     0.0 
        sad   124.0     0.0     0.0     0.0 
    disgust    54.0     0.0     0.0     0.0 
V_loss = 0.023732  Acc = 33.2378 

[ Epoch 2 ] T_Loss = 0.022434 
      t/p   neutral   happy     sad disgust 
    neutral   116.0     0.0     0.0     0.0 
      happy    55.0     0.0     0.0     0.0 
        sad   124.0     0.0     0.0     0.0 
    disgust    54.0     0.0     0.0     0.0 
V_loss = 0.023704  Acc = 33.2378 

[ Epoch 3 ] T_Loss = 0.022389 
      t/p   neutral   happy     sad disgust 
    neutral   116.0     0.0     0.0     0.0 
      happy    55.0     0.0     0.0     0.0 
        sad   124.0     0.0     0.0     0.0 
    disgust    54.0     0.0     0.0     0.0 
V_loss = 0.023671  Acc = 33.2378 

[ Epoch 4 ] T_Loss = 0.022300 
      t/p   neutral   happy     sad disgust 
    neutral   116.0     0.0     0.0     0.0 
     

      t/p   neutral   happy     sad disgust 
    neutral    36.0     9.0    49.0    22.0 
      happy    28.0     7.0    12.0     8.0 
        sad    15.0     2.0    12.0    95.0 
    disgust    22.0    11.0    12.0     9.0 
V_loss = 0.024348  Acc = 18.3381 

[ Epoch 30 ] T_Loss = 0.018668 
      t/p   neutral   happy     sad disgust 
    neutral    38.0     8.0    49.0    21.0 
      happy    34.0     4.0     9.0     8.0 
        sad    16.0     2.0    12.0    94.0 
    disgust    28.0     7.0    12.0     7.0 
V_loss = 0.024366  Acc = 17.4785 

[ Epoch 31 ] T_Loss = 0.018558 
      t/p   neutral   happy     sad disgust 
    neutral    41.0     8.0    44.0    23.0 
      happy    32.0     6.0     7.0    10.0 
        sad    15.0     2.0    10.0    97.0 
    disgust    27.0     8.0    10.0     9.0 
V_loss = 0.024461  Acc = 18.9112 

[ Epoch 32 ] T_Loss = 0.018571 
      t/p   neutral   happy     sad disgust 
    neutral    37.0     8.0    48.0    23.0 
      happy    32.0     3.0    12.